In [ ]:
import random
from typing import List, Tuple
import numpy as np

from deap import base, creator, tools, algorithms
from deap.tools import Logbook

In [ ]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

In [3]:
toolbox = base.Toolbox()

# Attribute generator
toolbox.register("attr_bool", random.randint, 0, 1)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, 100)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [4]:
# Evaluation Function
def evalOneMax(individual):
    return (sum(individual),)

In [12]:
# Operators
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)

In [9]:
def main():
    CROSSOVER_PROB = 0.5
    MUTATION_PROB = 0.2

    population = toolbox.population(n=300)

    # Evaluate the entire population
    fitnesses = list(map(toolbox.evaluate, population))
    for ind, fit in zip(population, fitnesses):
        ind.fitness.values = fit
    fits = [ind.fitness.values[0] for ind in population]
    generation = 0

    while max(fits) < 100 and generation < 1000:
        generation += 1
        print(f"--- Generation {generation} ---")
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))

        # Apply crossover on the offspring
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < CROSSOVER_PROB:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values

        # Apply mutation on the offspring
        for mutant in offspring:
            if random.random() < MUTATION_PROB:
                toolbox.mutate(mutant)
                del mutant.fitness.values

        # Evaluate the individuals with an invalid fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        population[:] = offspring

        # Gather all the fitnesses in one list and print the stats
        fits = [ind.fitness.values[0] for ind in population]
        mean = np.mean(fits)
        std = np.std(fits)
        min_fit = np.min(fits)
        max_fit = np.max(fits)

        print(f"  Min {min_fit}")
        print(f"  Max {max_fit}")
        print(f"  Avg {mean}")
        print(f"  Std {std}")

In [26]:
def GA() -> Tuple[List[creator.Individual], Logbook]:
    CROSSOVER_PROB = 0.5
    MUTATION_PROB = 0.2
    NUMBER_OF_GENERATIONS = 100

    population = toolbox.population(n=300)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values[0])
    stats.register("std", np.std)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    population, log = algorithms.eaSimple(
        population,
        toolbox,
        cxpb=CROSSOVER_PROB,
        mutpb=MUTATION_PROB,
        ngen=NUMBER_OF_GENERATIONS,
        stats=stats,
        halloffame=hof,
        verbose=True
    )

    return population, log

In [27]:
pop, log = GA()

gen	nevals	std    	avg  	min	max
0  	300   	4.82242	50.47	34 	63 
1  	187   	3.58268	54.38	46 	66 
2  	191   	3.03238	57.4633	48 	67 
3  	184   	3.10698	59.8   	52 	70 
4  	158   	3.03844	62.3233	54 	71 
5  	175   	2.95149	64.5733	54 	73 
6  	176   	2.61201	66.8067	60 	74 
7  	180   	2.74282	68.4833	59 	76 
8  	194   	2.7243 	70.3133	63 	78 
9  	174   	2.67016	72.06  	65 	80 
10 	176   	2.66145	73.6633	65 	82 
11 	172   	2.64381	75.3167	66 	83 
12 	176   	2.92247	76.85  	68 	87 
13 	180   	3.00384	78.6167	68 	88 
14 	204   	2.89905	80.2867	71 	89 
15 	177   	2.68584	82.14  	73 	90 
16 	170   	2.66692	83.7533	73 	90 
17 	176   	2.62356	85.06  	76 	91 
18 	177   	2.28157	86.4333	78 	92 
19 	172   	2.36835	87.53  	77 	94 
20 	191   	2.62903	88.1867	77 	94 
21 	188   	2.65452	89.3467	77 	95 
22 	190   	2.42308	90.6967	81 	96 
23 	194   	2.53832	91.6833	82 	97 
24 	202   	2.48743	92.2933	83 	96 
25 	168   	2.34278	93.2267	84 	97 
26 	202   	2.3348 	93.9067	84 	97 
27 	190   	2.21151	94.5767

In [28]:
log

[{'gen': 0,
  'nevals': 300,
  'std': np.float64(4.822423318899604),
  'avg': np.float64(50.47),
  'min': np.float64(34.0),
  'max': np.float64(63.0)},
 {'gen': 1,
  'nevals': 187,
  'std': np.float64(3.5826805606975345),
  'avg': np.float64(54.38),
  'min': np.float64(46.0),
  'max': np.float64(66.0)},
 {'gen': 2,
  'nevals': 191,
  'std': np.float64(3.0323789707459428),
  'avg': np.float64(57.46333333333333),
  'min': np.float64(48.0),
  'max': np.float64(67.0)},
 {'gen': 3,
  'nevals': 184,
  'std': np.float64(3.106981386061612),
  'avg': np.float64(59.8),
  'min': np.float64(52.0),
  'max': np.float64(70.0)},
 {'gen': 4,
  'nevals': 158,
  'std': np.float64(3.038440755094992),
  'avg': np.float64(62.32333333333333),
  'min': np.float64(54.0),
  'max': np.float64(71.0)},
 {'gen': 5,
  'nevals': 175,
  'std': np.float64(2.9514892662669285),
  'avg': np.float64(64.57333333333334),
  'min': np.float64(54.0),
  'max': np.float64(73.0)},
 {'gen': 6,
  'nevals': 176,
  'std': np.float64(2

In [ ]:
if __name__ == "__main__":
    main()

--- Generation 1 ---
  Min 40.0
  Max 66.0
  Avg 53.47666666666667
  Std 4.187615338378421
--- Generation 2 ---
  Min 46.0
  Max 68.0
  Avg 56.473333333333336
  Std 3.8169737867699447
--- Generation 3 ---
  Min 50.0
  Max 70.0
  Avg 59.776666666666664
  Std 3.698664924296634
--- Generation 4 ---
  Min 54.0
  Max 71.0
  Avg 62.71666666666667
  Std 3.2129512220940355
--- Generation 5 ---
  Min 58.0
  Max 73.0
  Avg 65.26333333333334
  Std 2.814365924245736
--- Generation 6 ---
  Min 57.0
  Max 76.0
  Avg 67.12333333333333
  Std 2.6909952723027137
--- Generation 7 ---
  Min 60.0
  Max 76.0
  Avg 68.91333333333333
  Std 2.688088953554592
--- Generation 8 ---
  Min 58.0
  Max 77.0
  Avg 70.53333333333333
  Std 2.7692758780751494
--- Generation 9 ---
  Min 65.0
  Max 79.0
  Avg 72.28666666666666
  Std 2.458554227363897
--- Generation 10 ---
  Min 65.0
  Max 80.0
  Avg 73.62333333333333
  Std 2.5089417866680144
--- Generation 11 ---
  Min 68.0
  Max 85.0
  Avg 75.27
  Std 2.242862754011786
--